In [ ]:
import pandas as pd
import json
from itertools import groupby

# Data Prep

In [ ]:
pd.set_option("display.max_rows", None) 

In [ ]:
# Input JSON data (from production)
config_path = '/src/shared/dataCatalog_config.json'

In [ ]:
with open(config_path, 'r') as f:
    data = json.load(f)

In [ ]:
# Data to drop
keys = [
    'popup', 
    'mode', 
    'numDecimal', 
    'visible', 
    'eaMetric', 
    'eaMetadata', 
    'opacity', 
    'url', 
    'eaDescription', 
    'tileLink', 
    'eaDfsLink', 
    'sourceType',
    'eaBCSDD',
    'autorefresh',
    'type', 
    'DownloadSource', 
    'eaLyrNum',
    'eaTags',
    'areaGeog',
    'eaScale',
    'tileURL',
    'hucNavStats',
    'hucNavStatsUnits',
    'layers',
    'cacheLevelNat',
    'agoID']

In [ ]:
# Drop all the keys from JSON dataset
for i in data:
    for key in keys:
        i.pop(key, None)

# Test process for subtopics with many layers

In [ ]:
# Filter data to eaID greater than or equal to 1000 (i.e. subtopics with layer references) 
data_filtered_subtopics = [d for d in data if d["eaID"] >= 1000]
print(data_filtered_subtopics)

In [ ]:
# Rename columns
rename_columns = {
    "categoryTab" : "category",
    "eaTopic" : "title",
    "SubLayerIds": "A",
    "SubLayerNames": "C"
}
filtered_subtopics_df = pd.DataFrame(data_filtered_subtopics)
filtered_subtopics_df = filtered_subtopics_df.rename(columns=rename_columns)
# filtered_subtopics_df = filtered_subtopics_df.set_index('eaID')
cols=['title', 'category', 'eaBC', 'eaCA', 'eaCPW', 'eaCS', 'eaFFM', 'eaNHM', 'eaRCA', 'name', 'A', 'C', ]
filtered_subtopics_df = filtered_subtopics_df[cols]
filtered_subtopics_df

In [ ]:
# Lump subtopics by eatopic
filtered_subtopics_df_carbon = filtered_subtopics_df.loc[filtered_subtopics_df['title'] == "Soils"]
filtered_subtopics_df_carbon

In [ ]:
explode = filtered_subtopics_df.explode(list('AC'))
layers = explode.iloc[:, 2:].reset_index(drop=True)
layers = layers.set_index(['name'])
cols = {
    "A": "eaID",
    "C": "title"
}
layers = layers.rename(columns=cols)
layers = layers.astype({'eaID': 'int32'})
layers

In [ ]:
layers.to_csv('/src/shared/layers.csv')

In [ ]:
subtopics = layers.to_json(orient='table')

In [ ]:
subtopics

In [ ]:
obj = json.loads(subtopics)
data = obj["data"]
data

In [ ]:
result = []
key_function = lambda x: (x['name'], x['eaBC'], x['eaCA'], x['eaCPW'], x['eaCS'], x['eaFFM'], x['eaNHM'], x['eaRCA'])
data.sort(key = key_function)
for group, checkings in groupby(data, key_function):
        user = {
                'name': group[0],
                'eaBC': group[1],
                'eaCA': group[2],
                'eaCPW': group[3],
                'eaCS': group[4],
                'eaFFM': group[5],
                'eaNHM': group[6],
                'eaRCA': group[7],
                'layers': []
        }
        for title in checkings:
                user['layers'].append(({"title": title['title'], "eaID": title['eaID']}))
        result.append(user)

print(json.dumps(result, indent=2))

# Formalize Process

In [ ]:
def process_old_json(subtopic):
    filtered_subtopics_df_carbon = filtered_subtopics_df.loc[filtered_subtopics_df['title'] == subtopic]
    explode = filtered_subtopics_df_carbon.explode(list('AC'))
    layers = explode.iloc[:, 2:].reset_index(drop=True)
    layers = layers.set_index(['name'])
    cols = {
        "A": "eaID",
        "C": "title"
    }
    layers = layers.rename(columns=cols)
    layers = layers.astype({'eaID': 'int32'})
    subtopics = layers.to_json(orient='table')
    obj = json.loads(subtopics)
    data = obj["data"]
    result = []
    key_function = lambda x: (x['name'], x['eaBC'], x['eaCA'], x['eaCPW'], x['eaCS'], x['eaFFM'], x['eaNHM'], x['eaRCA'])
    data.sort(key = key_function)
    for group, checkings in groupby(data, key_function):
            user = {
                    'name': group[0],
                    'eaBC': group[1],
                    'eaCA': group[2],
                    'eaCPW': group[3],
                    'eaCS': group[4],
                    'eaFFM': group[5],
                    'eaNHM': group[6],
                    'eaRCA': group[7],
                    'layers': []
            }
            for title in checkings:
                    user['layers'].append(({"title": title['title'], "eaID": title['eaID']}))
            result.append(user)

    print(json.dumps(result, indent=2))

### Gather data...brute force copy and paste as "subtopic"...

In [ ]:
process_old_json('Carbon Storage')

In [ ]:
process_old_json('Crop Productivity')

In [ ]:
process_old_json('Ecosystem Markets')

In [ ]:
process_old_json('Energy Potential')

In [ ]:
process_old_json('Engagement with Outdoors')

In [ ]:
process_old_json('Health and Economic Outcomes')

In [ ]:
process_old_json('Land Cover: Near-Water')

In [ ]:
process_old_json('Land Cover: Type')

In [ ]:
process_old_json('Landscape Pattern')

In [ ]:
process_old_json('Livestock and Poultry Production')

In [ ]:
process_old_json('Near-Road Environments')

In [ ]:
process_old_json('Pollutant Reduction: Air')

In [ ]:
process_old_json('Pollutant Reduction: Water')

In [ ]:
process_old_json('Soils')

In [ ]:
process_old_json('Species: At-Risk and Priority')

In [ ]:
process_old_json('Species: Other')

In [ ]:
process_old_json('Water Supply, Runoff, and Flow')

In [ ]:
process_old_json('Water Use')

In [ ]:
process_old_json('Weather and Climate')

In [ ]:
process_old_json('Wetlands and Lowlands')

In [ ]:
process_old_json('Harmful Algal Blooms')

In [ ]:
process_old_json('Impaired Waters')

In [ ]:
process_old_json('National Air Toxics Assesment')

In [ ]:
process_old_json('Pollutants: Nutrients')

In [ ]:
process_old_json('Pollutants: Other')

In [ ]:
process_old_json('Sites Reporting to EPA')

In [ ]:
process_old_json('Population Distribution')

In [ ]:
process_old_json('Hydrologic Features')

# TO DO: formalize this bit

In [ ]:
new_df = explode.iloc[:, :2]
new_df = new_df.drop_duplicates()
new_df

In [ ]:
# Create DataCatologItem object
# Define a custom function to create a nested structure
def custom_nested_structure(row):
    return {'Person': {'Name': row['Name'], 'Age': row['Age']}}

In [ ]:
output_config_path = '/src/shared/dataCatalog_testInitSubtopicsOnly.json'

In [ ]:
# Save Subtopic JSON only
with open(output_config_path, 'w') as file:
    json.dump(data_filtered_subtopics, file)

# TO DO: subtopic with 1 layer

In [ ]:
# Filter data to eaID less than 1000 (i.e. layers) 
data_filtered_layers = [d for d in data if d["eaID"] < 1000]
print(data_filtered_layers)

In [ ]:
# Filter out where "IsSubLayer" is true (these are retained in the subtopic data)
filtered_layers_df = pd.DataFrame(data_filtered_layers)
filtered_layers_df.IsSubLayer = filtered_layers_df.IsSubLayer.fillna(False)
notSubLayers_dict = filtered_layers_df.loc[filtered_layers_df['IsSubLayer'] == False].to_dict('records')
notSubLayers_dict

In [ ]:
output_config_path = '/src/shared/dataCatalog_testInitLayersOnly.json'

In [ ]:
# Save Layer JSON only
with open(output_config_path, 'w') as file:
    json.dump(notSubLayers_dict, file)